In [1]:
%pip install openai
%pip install anthropic
%pip install pandas
%pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
from dotenv import load_dotenv

load_dotenv()

lang_codes = {
    "zgh": "Standard Moroccan Tamazight",
    "eng": "English"
}

# Load FLORES devtest data

In [3]:
import pandas as pd

# Paths to original FLORES devtest data
zgh_path = "devtest.zgh_Tfng"
eng_path = "devtest.eng_Latn"

In [4]:
def load_floresp(zgh_path, eng_path):
    zgh_lines = []
    eng_lines = []
    with open(zgh_path, 'r') as zgh_file:
        for line in zgh_file:
            zgh_lines.append(line.strip())
    with open(eng_path, 'r') as eng_file:
        for line in eng_file:
            eng_lines.append(line.strip())

    return zgh_lines, eng_lines

zgh_lines, eng_lines = load_floresp(zgh_path, eng_path)

original_data = pd.DataFrame({"eng": eng_lines, "zgh": zgh_lines})
original_data

,eng,zgh
0,"""We now have 4-month-old mice that are non-dia...",“ⵏⵍⴰ ⴷⵖⵉ ⵉⵖⵔⴷⴰⵢⵏ ⵏ 4 ⵏ ⵉⵢⵢⵉⵔⵏ ⴷ ⵓⵔ ⵉⵍⵍⵉ ⵡⴰⵟⵟⴰⵏ...
1,"Dr. Ehud Ur, professor of medicine at Dalhousi...","ⴰⴷⵓⴽⵜⵓⵔ ⵉⵀⵓⵔ ⵓⵔ, ⴰⵙⵍⵎⴰⴷ ⵏ ⵜⴰⵙⵏⵓⵊⵢⴰ ⴳ ⵜⵙⴷⴰⵡⵉⵜ ⵏ..."
2,"Like some other experts, he is skeptical about...","ⴰⵎⵎ ⴽⵔ ⵏ ⵡⵉⵢⵢⴰⴹ ⵉⵎⵓⵣⴰⵢⵏ, ⵉⴳⴰ ⴰⵎⵛⴽⴰⴽ ⴳ ⵓⵢⵏⵏⴰ ⵉⵇ..."
3,"On Monday, Sara Danius, permanent secretary of...","ⴳ ⵓⵢⵏⴰⵙ, ⵜⵏⵏⴰ ⵚⴰⵕⴰ ⴷⴰⵏⵢⵓⵙ, ⵜⴰⵏⴼⵍⵓⵙⵜ ⵜⴰⵎⵖⵍⴰⵍⵜ ⵏ..."
4,"Danius said, ""Right now we are doing nothing. ...","ⵜⵏⵏⴰ ⴷⴰⵏⵢⵓⵙ, “ⴷⵖⵉ ⵓⵔ ⴷⴰ ⵏⵙⵙⴽⴰⵔ ⴰⵎⵢⴰ. ⵖⵔⵉⵖ ⴰⵙ ⴰ..."
...,...,...
1007,"As the areas are sparsely populated, and light...","ⵍⵍⵉⴳ ⴷⵔⵓⵙⵏ ⵉⵎⵣⴷⴰⵖⵏ ⴳ ⵜⵎⵏⴰⴹⵜ, ⴷ ⵡⵓⵍⵓⴼ ⵉⴷⵔⵓⵙⵏ ⵛⵉ..."
1008,Japanese work culture is more hierarchical and...,ⵜⴳⴰ ⵜⴰⴷⵍⵙⴰ ⵏ ⵜⵡⵓⵔⵉ ⵜⴰⵢⴰⴱⴰⵏⵉⵢⵜ ⴷ ⵜⴰⵎⵓⵣⴳⴰⵏⵜ ⵓⴳⴳⴰ...
1009,"Suits are standard business attire, and cowork...","ⴳⴰⵏ ⵉⴽⵓⵙⵜⵉⵎⵏ ⵜⵉⵎⵍⵙⴰ ⵜⵓⵏⵚⵉⴱⵉⵏ ⵏ ⵉⴷ ⴱⴰⴱ ⵏⵜⵡⵓⵔⵉ, ..."
1010,"Workplace harmony is crucial, emphasizing grou...","ⵜⵙⵜⴰⵡⵀⵎⵎⴰ ⵜⵎⵙⴰⵙⵉⵜ ⵛⵉⴳⴰⵏ ⴳ ⵡⴰⵏⵙ ⵏ ⵜⵡⵓⵔⵉ, ⴰⵔ ⴷ ⵉ..."


In [5]:
corrected_data = pd.read_csv("Corrected_FLORES_devtest.csv")
corrected_data

,#Sent.,English,Tamazight (original),Tamazight (Corrected),Notes
0,1,"""We now have 4-month-old mice that are non-dia...",“ⵏⵍⴰ ⴷⵖⵉ ⵉⵖⵔⴷⴰⵢⵏ ⵏ 4 ⵏ ⵉⵢⵢⵉⵔⵏ ⴷ ⵓⵔ ⵉⵍⵍⵉ ⵡⴰⵟⵟⴰⵏ...,“ⵏⵍⴰ ⴷⵖⵉ ⵉⵖⵔⴷⴰⵢⵏ ⵏ 4 ⵡⴰⵢⵢⵓⵔⵏ ⵓⵔ ⵓⴹⵉⵏⵏ ⵙ ⵜⵎⴰⴹⵓⵏ...,NaN
1,2,"Dr. Ehud Ur, professor of medicine at Dalhousi...","ⴰⴷⵓⴽⵜⵓⵔ ⵉⵀⵓⵔ ⵓⵔ, ⴰⵙⵍⵎⴰⴷ ⵏ ⵜⴰⵙⵏⵓⵊⵢⴰ ⴳ ⵜⵙⴷⴰⵡⵉⵜ ⵏ...","ⵉⵏⵏⴰ ⵓⴷⵓⴽⵜⵓⵔ ⵉⵀⵓⴷ ⵓⵔ, ⴰⵙⵍⵎⴰⴷ ⵏ ⵜⵙⵏⵉⵊⵊⵉⵜ ⴳ ⵜⵙⴷⴰ...",NaN
2,3,"Like some other experts, he is skeptical about...","ⴰⵎⵎ ⴽⵔ ⵏ ⵡⵉⵢⵢⴰⴹ ⵉⵎⵓⵣⴰⵢⵏ, ⵉⴳⴰ ⴰⵎⵛⴽⴰⴽ ⴳ ⵓⵢⵏⵏⴰ ⵉⵇ...","ⴰⵎⵎ ⴽⵔⴰ ⵏ ⵉⵎⵓⵣⴰⵢⵏ ⵢⴰⴹⵏⵉⵏ, ⵉⴳⴰ ⴰⵎⵛⴽⴰⴽ ⴳ ⵓⵢⵏⵏⴰ ⵉ...",NaN
3,4,"On Monday, Sara Danius, permanent secretary of...","ⴳ ⵓⵢⵏⴰⵙ, ⵜⵏⵏⴰ ⵚⴰⵕⴰ ⴷⴰⵏⵢⵓⵙ, ⵜⴰⵏⴼⵍⵓⵙⵜ ⵜⴰⵎⵖⵍⴰⵍⵜ ⵏ...","ⴳ ⵓⵢⵏⴰⵙ, ⵜⵏⵏⴰ ⵚⴰⵕⴰ ⴷⴰⵏⵢⵓⵙ, ⵜⴰⵏⴼⵍⵓⵙⵜ ⵜⴰⵎⵖⵍⴰⵍⵜ ⵏ...",NaN
4,5,"Danius said, ""Right now we are doing nothing. ...","ⵜⵏⵏⴰ ⴷⴰⵏⵢⵓⵙ, “ⴷⵖⵉ ⵓⵔ ⴷⴰ ⵏⵙⵙⴽⴰⵔ ⴰⵎⵢⴰ. ⵖⵔⵉⵖ ⴰⵙ ⴰ...","ⵜⵏⵏⴰ ⴷⴰⵏⵢⵓⵙ, “ⴷⵖⵉ ⵓⵔ ⴷⴰ ⵏⵙⴽⴰⵔ ⴰⵎⵢⴰ. ⵖⵔⵉⵖ ⴰⵙ ⴰⵣ...",NaN
...,...,...,...,...,...
1007,1008,"As the areas are sparsely populated, and light...","ⵍⵍⵉⴳ ⴷⵔⵓⵙⵏ ⵉⵎⵣⴷⴰⵖⵏ ⴳ ⵜⵎⵏⴰⴹⵜ, ⴷ ⵡⵓⵍⵓⴼ ⵉⴷⵔⵓⵙⵏ ⵛⵉ...","ⵍⵍⵉⴳ ⴷⵔⵓⵙⵏ ⵉⵎⵣⴷⴰⵖⵏ ⴳ ⵜⵎⵏⴰⴹⵜ, ⴷ ⵡⵓⵍⵓⴼ ⵉⴷⵔⵓⵙⵏ ⵛⵉ...",NaN
1008,1009,Japanese work culture is more hierarchical and...,ⵜⴳⴰ ⵜⴰⴷⵍⵙⴰ ⵏ ⵜⵡⵓⵔⵉ ⵜⴰⵢⴰⴱⴰⵏⵉⵢⵜ ⴷ ⵜⴰⵎⵓⵣⴳⴰⵏⵜ ⵓⴳⴳⴰ...,ⵜⴳⴰ ⵜⴰⴷⵍⵙⴰ ⵏ ⵜⵡⵓⵔⵉ ⵜⴰⵢⴰⴱⴰⵏⵉⵢⵜ ⴷ ⵜⴰⵎⵓⵣⴳⴰⵏⵜ ⴷ ⵜⵓ...,NaN
1009,1010,"Suits are standard business attire, and cowork...","ⴳⴰⵏ ⵉⴽⵓⵙⵜⵉⵎⵏ ⵜⵉⵎⵍⵙⴰ ⵜⵓⵏⵚⵉⴱⵉⵏ ⵏ ⵉⴷ ⴱⴰⴱ ⵏⵜⵡⵓⵔⵉ, ...","ⴳⴰⵏ ⵉⴽⵓⵙⵜⵉⵎⵏ ⵜⵉⵎⵍⵙⴰ ⵜⵓⵏⵚⵉⴱⵉⵏ ⵏ ⵉⴷ ⴱⴰⴱ ⵏⵜⵡⵓⵔⵉ, ...",NaN
1010,1011,"Workplace harmony is crucial, emphasizing grou...","ⵜⵙⵜⴰⵡⵀⵎⵎⴰ ⵜⵎⵙⴰⵙⵉⵜ ⵛⵉⴳⴰⵏ ⴳ ⵡⴰⵏⵙ ⵏ ⵜⵡⵓⵔⵉ, ⴰⵔ ⴷ ⵉ...","ⵜⵅⴰⵜⵔ ⴽⵉⴳⴰⵏ ⵜⵎⵙⴰⵙⵉⵜ ⴳ ⵡⴰⵏⵙⴰ ⵏ ⵜⵡⵓⵔⵉ, ⵙ ⵓⵙⵎⵙⵙⵉ ...",NaN


In [6]:
# Sanity check
assert original_data['eng'].equals(corrected_data['English'])

In [7]:
# convert the corrected data to a dictionary
data = corrected_data[['English', 'Tamazight (Corrected)']].to_dict('list')
# rename the columns
data = {'eng': data['English'], 'zgh': data['Tamazight (Corrected)']}

# Define the GPT and Claude models used for evaluation

In [8]:
from openai import OpenAI
from abc import ABC, abstractmethod
import socket

SYSTEM_PROMPT = "You are an expert linguist and translator from and into Standard Moroccan Tamazight. Only use Tifinagh when translating into Tamazight. Don't use the Latin alphabet. When asked to translate a text, don't include a prelude. Just provide the translation and nothing else."

class LLMInterface(ABC):
    """
    This is the Large Language Model (LLM) interface. It provides a common interface for different language models.
    Each model should implement the 'get_result' method, which takes a prompt and returns the LLM result.
    """
    name = None
    stop = "\n" # Stop sequence for the LLM (when to stop generating text)
    temperature = 0.7
    max_tokens = 500

    @abstractmethod
    def get_result(self, prompt) -> str:
        raise NotImplementedError

In [9]:
class GPT(LLMInterface):
    """
    OpenAI is a general class for OpenAI's models, which other specific GPT model classes inherit from.
    """
    def __init__(self):
        # Initialize LLM specifics
        self.client = OpenAI(
            base_url=f"http://{socket.gethostname()}.local:1234/v1",
            api_key="lm-studio"
            )
        self.model = None

    def get_result(self, prompt):
        try:
            response = self.client.chat.completions.create(
                model=self.model,
                messages=self.create_messages_from_prompt(prompt),
                max_tokens=self.max_tokens,
                temperature=self.temperature,
                stop=self.stop
                )
            result = response.choices[0].message.content.strip()
        except Exception as e:
            print(f"Error: {e}")
            result = None
        return result

    def create_messages_from_prompt(self, prompt):
        messages = [
            {"role": "system", "content": SYSTEM_PROMPT},
            {
                "role": "user",
                "content": prompt
                }
            ]
        return messages

In [10]:
class GPT_4o(GPT):
    """
    Class for the GPT-4o model. Inherits from GPT.
    """
    name = "GPT-4o"

    def __init__(self):
        super().__init__()
        self.model = "gpt-4o"

In [11]:
import anthropic


class Claude(LLMInterface):
    """
    Claude is a general class for models from Anthropic, which other specific Claude model classes inherit from.
    """
    def __init__(self):
        # Initialize LLM specifics
        self.client = anthropic.Anthropic()
        self.model = None

    def get_result(self, prompt):
        try:
            response = self.client.messages.create(
                model=self.model,
                system=SYSTEM_PROMPT,
                messages=self.create_messages_from_prompt(prompt),
                max_tokens=self.max_tokens,
                temperature=self.temperature,
                )
            result = response.content[0].text.strip()
        except Exception as e:
            print("Error:", e)
            result = None
        return result

    def create_messages_from_prompt(self, prompt):
        messages = [
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": prompt
                    }
                ]
            }
        ]
        return messages

In [12]:
class Sonnet_3_5(Claude):
    """
    Class for the Claude 3.5 Sonnet model. Inherits from Claude.
    """
    name = "Sonnet 3.5"

    def __init__(self):
        super().__init__()
        self.model = "claude-3-5-sonnet-20240620"

# Get the translations from the models

In [13]:
from tqdm import tqdm

PROMPT_TEMPLATE = "Translate the following {source} text to {target}:\n{source}: {source_text}\n{target}: "

def get_translations(source_code, target_code, data, model, save_to_txt=True):
    source = lang_codes[source_code]
    target = lang_codes[target_code]

    translations = data.copy()
    translations['Translation'] = []
    source_texts = data[source_code]
    filename = f"{model.model}_{source_code}_to_{target_code}.txt"

    for source_text in tqdm(source_texts):
        prompt = PROMPT_TEMPLATE.format(source=source, target=target, source_text=source_text)
        translation = model.get_result(prompt)
        translations['Translation'].append(translation)
        if save_to_txt:
            with open(filename, 'a') as f:
                if translation:
                    f.write(translation + "\n")
                else:
                    f.write("\n")

    translations = pd.DataFrame(translations)
    # replace "source_code" with "Source" and "target_code" with "Target" in the column names
    translations = translations.rename(columns={source_code: "Source", target_code: "Target"})
    # reorder the columns to have "Source", "Target", and "Translation" as the first three columns
    translations = translations[["Source", "Target", "Translation"]]

    return translations

In [14]:
# Initialize the Claude Sonnet 3.5 model
sonnet_3_5 = Sonnet_3_5()

eng_to_zgh = get_translations("eng", "zgh", data, sonnet_3_5)
eng_to_zgh

  0%|          | 0/1012 [00:00<?, ?it/s]

  6%|▋         | 65/1012 [06:42<47:33,  3.01s/it]  

Error: Error code: 400 - {'type': 'error', 'error': {'type': 'invalid_request_error', 'message': 'Your credit balance is too low to access the Claude API. Please go to Plans & Billing to upgrade or purchase credits.'}}
Error: Error code: 400 - {'type': 'error', 'error': {'type': 'invalid_request_error', 'message': 'Your credit balance is too low to access the Claude API. Please go to Plans & Billing to upgrade or purchase credits.'}}


  7%|▋         | 67/1012 [06:42<24:54,  1.58s/it]

Error: Error code: 400 - {'type': 'error', 'error': {'type': 'invalid_request_error', 'message': 'Your credit balance is too low to access the Claude API. Please go to Plans & Billing to upgrade or purchase credits.'}}
Error: Error code: 400 - {'type': 'error', 'error': {'type': 'invalid_request_error', 'message': 'Your credit balance is too low to access the Claude API. Please go to Plans & Billing to upgrade or purchase credits.'}}


...

 17%|█▋        | 172/1012 [07:06<02:46,  5.05it/s]

Error: Error code: 400 - {'type': 'error', 'error': {'type': 'invalid_request_error', 'message': 'Your credit balance is too low to access the Claude API. Please go to Plans & Billing to upgrade or purchase credits.'}}
Error: Error code: 400 - {'type': 'error', 'error': {'type': 'invalid_request_error', 'message': 'Your credit balance is too low to access the Claude API. Please go to Plans & Billing to upgrade or purchase credits.'}}


 17%|█▋        | 173/1012 [07:07<34:31,  2.47s/it]

Error: Error code: 400 - {'type': 'error', 'error': {'type': 'invalid_request_error', 'message': 'Your credit balance is too low to access the Claude API. Please go to Plans & Billing to upgrade or purchase credits.'}}


KeyboardInterrupt: 

In [ ]:
zgh_to_eng = get_translations("zgh", "eng", data, sonnet_3_5)
zgh_to_eng

100%|██████████| 3/3 [00:07<00:00,  2.50s/it]


,Source,Target,Translation
0,“ⵏⵍⴰ ⴷⵖⵉ ⵉⵖⵔⴷⴰⵢⵏ ⵏ 4 ⵡⴰⵢⵢⵓⵔⵏ ⵓⵔ ⵓⴹⵉⵏⵏ ⵙ ⵜⵎⴰⴹⵓⵏ...,"""We now have 4-month-old mice that are non-dia...","""We now have 4-month-old mice that are not aff..."
1,"ⵉⵏⵏⴰ ⵓⴷⵓⴽⵜⵓⵔ ⵉⵀⵓⴷ ⵓⵔ, ⴰⵙⵍⵎⴰⴷ ⵏ ⵜⵙⵏⵉⵊⵊⵉⵜ ⴳ ⵜⵙⴷⴰ...","Dr. Ehud Ur, professor of medicine at Dalhousi...","Dr. Ehud Ur, a professor of endocrinology at D..."
2,"ⴰⵎⵎ ⴽⵔⴰ ⵏ ⵉⵎⵓⵣⴰⵢⵏ ⵢⴰⴹⵏⵉⵏ, ⵉⴳⴰ ⴰⵎⵛⴽⴰⴽ ⴳ ⵓⵢⵏⵏⴰ ⵉ...","Like some other experts, he is skeptical about...","Like some other scientists, he is skeptical ab..."
